Introduccion al problema

In [1]:
# %pip install gymnasium

### Windows:

method 1

In [2]:
# %pip install swig

In [3]:
# %pip install gymnasium[box2d]

method 2

In [4]:
# %pip install swig

In [5]:
# %pip install ufal.pybox2d

In [6]:
# %pip install pygame

### Linux:

In [7]:
# %pip install gymnasium[box2d]

https://gymnasium.farama.org/environments/box2d/lunar_lander/

In [8]:
from lunar import LunarLanderEnv

Tensorflow or Torch

In [9]:
import torch


In [10]:
# Initialize the environment
lunar = LunarLanderEnv()
print(type(lunar.env.observation_space))
print(type(lunar.env.action_space))

<class 'gymnasium.spaces.box.Box'>
<class 'gymnasium.spaces.discrete.Discrete'>


El espacio de acciones es un valor del 0 al 3 que indica que acciones tomará el modulo lunar para esa iteración.

en concreto son las siguientes:

|value| action                        |
|-----|-------------------------------|
| 0   | do nothing                    |
| 1   | fire left orientation engine  |
| 2   | fire main engine              |
| 3   | fire right orientation engine |


In [11]:
lunar.env.action_space

Discrete(4)

El espacio de observaciones son un conjunto de valores flotantes y booleanos que indica el estado del modulo lunar.

en concreto son las siguientes:

|value| observation                               |
|-----|-------------------------------------------|
| 0   | coordenada X (float)                      |
| 1   | coordenada Y (float)                      |
| 2   | velocidad lineal X (float)                |
| 3   | velocidad lineal Y (float)                |
| 4   | Angulo en radianes desde -2π a +2π (float)|
| 5   | Velocidad angula (float)                  |
| 6   | Contacto de la pierna Izquierda (bool)    |
| 7   | Contacto de la pierna Derecha (bool)      |

In [12]:
# se muestran los valores minimos y maximos del espacio de observaciones.
lunar.env.observation_space

Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)

In [13]:
observation_count = lunar.env.observation_space.shape[0] 
action_count = lunar.env.action_space.n

print(f"observations: {observation_count}, actions: {action_count}")

observations: 8, actions: 4


In [14]:
#valores minimos y maximos para las observaciones.
print(lunar.env.observation_space.low) 
print(lunar.env.observation_space.high)

[ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ]
[ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ]


Sample ofrece una combinacion aleatoria del conjunto.

In [15]:
print(lunar.env.action_space.sample())  # Take a random action

3


In [16]:
print(lunar.env.observation_space.sample())  # Sample a random observation

[-0.3311917  -1.7676712   6.937776   -4.45337    -2.1596286   8.358777
  0.9912258   0.27499476]


Running a random episode.

In [17]:
def test_lunar_lander(steps_to_run_before_pause, agent, episodes=1):
    """
    Test the Lunar Lander environment with a given agent.
    
    Parameters:
    steps_to_run_before_pause (int): Number of steps to run before pausing for user input.
    agent: The agent to be tested in the environment.
    
    Returns:
    None
    """
    # Initialize the environment
    lunar = LunarLanderEnv(render_mode=None)
    score_total = []
    if(agent is not None):
        # Set the agent's environment
        agent.lunar = lunar
        
    for _ in range(episodes):
        counter, score = 0, 0

        while True:
            if steps_to_run_before_pause != 0 and counter % steps_to_run_before_pause == 0:
                input("Press Enter to continue...")

            if(agent is not None):
                _, reward, done, action = agent.act()
                
            else:
                # Sample a random action from the action space
                action = lunar.env.action_space.sample()
            
                # Take a step in the environment
                _, reward, done = lunar.take_action(action, verbose=True)
                
            score += reward
            
            counter += 1
            
            if done:
                print(f"Episode finished, score: {score}")
                score_total.append(score)
                break
        if(agent is not None):
            # Reset the agent's environment for the next episode
            agent.lunar.reset()
        else:
            # Reset the environment for the next episode
            lunar.reset()
        
    # Close the environment
    print(f"Total score over {episodes} episodes: {sum(score_total)/len(score_total)}")
    lunar.close()

In [18]:
test_lunar_lander(steps_to_run_before_pause=0, agent=None, episodes=1)

Step taken: 0, New state: [-0.01308823  1.4305828  -0.66194016  0.42404655  0.01500874  0.14839913
  0.          0.        ], Reward: -0.2097733466805778, Done: False
Step taken: 1, New state: [-0.01972141  1.4395199  -0.67306185  0.39707723  0.02465094  0.19286183
  0.          0.        ], Reward: -1.4299170163593817, Done: False
Step taken: 0, New state: [-0.02635488  1.4478581  -0.6730912   0.37039977  0.03429082  0.19281535
  0.          0.        ], Reward: -0.4896929596471864, Done: False
Step taken: 2, New state: [-0.03306446  1.4563845  -0.6804208   0.37871203  0.04365255  0.18725197
  0.          0.        ], Reward: -3.1461651315636514, Done: False
Step taken: 0, New state: [-0.03977442  1.4643121  -0.6804477   0.3520309   0.053013    0.18722644
  0.          0.        ], Reward: -0.4855597806811147, Done: False
Step taken: 2, New state: [-0.04653168  1.4724548  -0.68507826  0.3615468   0.06228516  0.18546005
  0.          0.        ], Reward: -2.912096704843253, Done: False

DQN

In [19]:
from DQN import DQNAgent
lunar = LunarLanderEnv(render_mode=None)

Train

In [ ]:
agent = DQNAgent(lunar)
agent.train()

Using device: cpu


Test

In [ ]:
# agent with epsilon = 0.0 (no exploration)
agent = DQNAgent(lunar, epsilon=0.0)
agent.load_model("modelo_DQN.h5")

Using device: cpu
QNetwork:
 DQN(
  (model): Sequential(
    (0): Linear(in_features=8, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=4, bias=True)
  )
)
Model loaded from modelo_DQN.h5


In [ ]:
test_lunar_lander(steps_to_run_before_pause=0, agent=agent, episodes=100)

Episode finished, score: -26.94699531287399
Episode finished, score: -24.207171171179876
Episode finished, score: -32.931736018297656
Episode finished, score: -9.815367005881512
Episode finished, score: 3.6127161852417675
Episode finished, score: 7.233316595495387
Episode finished, score: -1.7151160877529266
Episode finished, score: -1.1398311757982882
Episode finished, score: -34.259377430174546
Episode finished, score: -14.539112834668106
Episode finished, score: -2.4748684326585857
Episode finished, score: -15.152124734064628
Episode finished, score: -41.58915296138186
Episode finished, score: -19.068839891689525
Episode finished, score: -37.56357135395363
Episode finished, score: -19.003499039106316
Episode finished, score: -16.91249309281332
Episode finished, score: -0.5925008532810949
Episode finished, score: -25.24383622027657
Episode finished, score: 19.956399577751736
Episode finished, score: -30.433635860197487
Episode finished, score: -6.813149603464224
Episode finished, sco

REINFORCE

In [ ]:
from REINFORCE import REINFORCEAgent
lunar = LunarLanderEnv(render_mode=None)

ModuleNotFoundError: No module named 'REINFORCE'

In [ ]:
# agent = REINFORCEAgent(lunar, episodes=5000)
# agent.load_model("modelo_REINFORCE.h5")

In [ ]:
agent = REINFORCEAgent(lunar)
agent.load_model("modelo_REINFORCE.h5")

e:\Discovirtual-us\IA personal\Aprendizaje por refuerzo trabajo 2024-2025\VersionAlumnos\REINFORCE.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.actor_net.load_s

In [ ]:
test_lunar_lander(steps_to_run_before_pause=75, agent=agent, episodes=1)

Episode finished, score: 264.42781863081314
Environment closed.
